<a href="https://colab.research.google.com/github/trappertwo/rPPG_experiments/blob/main/compress_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### UBFC-rPPG dataset in drive folder
### (TODO) Figure out how to import data set directly into colab
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
### Compress the UBFC dataset and store the compressed files under a separate
### dir structure. Only needs to be run once.
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import shutil
import os

!pip install -q mediapy
import mediapy as media

base_path = '/content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_2' # @param {type:"string"}
compressed_path = '/content/drive/MyDrive/research_project/data/UBFC-rPPG/500_1_train' # @param {type:"string"}

!ls -l $base_path/subject9
!ls -l $compressed_path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.2 MB/s eta 0:00:00
total 1814623
-r-------- 1 root root      96774 Mar  7  2018 ground_truth.txt
-r-------- 1 root root 1858075976 Nov  2  2017 vid.avi
ls: cannot access '/content/drive/MyDrive/research_project/data/UBFC-rPPG/500_1_train': No such file or directory


In [4]:
COMPRESSION_RATIO = 500 # @param {type:"number"}
LENGTH = 640 # @param {type:"number"}
WIDTH = 480 # @param {type:"number"}
FPS = 30 # @param {type:"number"}
ORIGINAL_BIT_RATE = LENGTH*WIDTH*8*3*FPS #Only for UBFC-rPPG
print("Original bit rate: " + str(ORIGINAL_BIT_RATE))
COMPRESSION_BIT_RATE = ORIGINAL_BIT_RATE // COMPRESSION_RATIO
print("Compression bit rate: " + str(COMPRESSION_BIT_RATE))

Original bit rate: 221184000
Compression bit rate: 442368


In [ ]:
# Compress data - creates new dataset
#BIT_RATE = 5000000 # @param {type:"number"}
dirnames = os.listdir(base_path)
if not os.path.isdir(compressed_path):
  os.mkdir(compressed_path)
for dirname in dirnames:
  if not os.path.isdir(os.path.join(compressed_path, dirname)):
    os.mkdir(os.path.join(compressed_path, dirname))
  for filename in os.listdir(os.path.join(base_path, dirname)):
    if filename.endswith('.avi'):
      video = media.read_video(os.path.join(base_path, dirname, filename))
      compressed_data = media.compress_video(video, bps=COMPRESSION_BIT_RATE, fps=30)
      print("Writing " + os.path.join(compressed_path, dirname, 'vid.mp4'))
      with open(os.path.join(compressed_path, dirname, 'vid.mp4'), 'wb') as f:
        f.write(compressed_data)
    else:
      shutil.copy(os.path.join(base_path, dirname, filename), os.path.join(compressed_path, dirname, filename))
print("All files compressed")

Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/500_1_train/subject1/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/500_1_train/subject10/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/500_1_train/subject11/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/500_1_train/subject12/vid.mp4


In [ ]:
import cv2

def read_video(video_file):
        """Reads a video file, returns frames(T, H, W, 3) """
        VidObj = cv2.VideoCapture(video_file)
        VidObj.set(cv2.CAP_PROP_POS_MSEC, 0)
        success, frame = VidObj.read()
        frames = list()
        while success:
            frame = cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2RGB)
            frame = np.asarray(frame)
            frames.append(frame)
            success, frame = VidObj.read()
        return np.asarray(frames)

#original_path = base_path+'/subject1/vid.avi'
#!ls -l $original_path
#original_vid = read_video(original_path)
#print('original frames:')
#len(original_vid)

new_file = '/content/drive/MyDrive/research_project/data/UBFC-rPPG/500_1_train/subject1/vid.mp4' # @param {type:"string"}
!ls -l $new_file
new_vid = media.read_video(new_file)
#new_vid = read_video(new_file)

print('new frames:')
len(new_vid)

#new_vid = read_video(compressed_path+'/subject1/vid.mp4')
#print(original_vid.size / new_vid.size)
#len(new_vid)